## Available Pretrained Models

In [1]:
from chgnet.model.model import CHGNet

# Load the latest CHGNet model (default: 0.3.0)
chgnet = CHGNet.load()
# Load specific CHGNet versions
chgnet = CHGNet.load(model_name='r2scan')

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet vr2scan initialized with 412,525 parameters
CHGNet will run on cuda


## Direct Inference (Static Calculation)

In [2]:
from chgnet.model.model import CHGNet
from pymatgen.core import Structure

chgnet = CHGNet.load()
structure = Structure.from_file('../example_crystal.cif')
prediction = chgnet.predict_structure(structure)

for key, unit in [
    ("energy", "eV/atom"),
    ("forces", "eV/A"),
    ("stress", "GPa"),
    ("magmom", "mu_B"),
]:
    print(f"CHGNet-predicted {key} ({unit}):\n{prediction[key[0]]}\n")

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet-predicted energy (eV/atom):
-8.108797073364258

CHGNet-predicted forces (eV/A):
[[-3.9339066e-06 -1.9352883e-06 -3.0980445e-06]
 [-2.7120113e-06  1.3895333e-06  6.5192580e-07]
 [-1.5497208e-06 -1.0030344e-06  1.4491379e-06]
 [-1.4171004e-05 -4.8005022e-06 -3.8337894e-06]
 [-1.8921375e-02 -5.8865808e-02  2.5192123e-02]
 [-1.9067459e-02 -5.8970477e-02  2.5044795e-02]
 [ 1.8944174e-02  5.8859963e-02 -2.5182815e-02]
 [ 1.9067153e-02  5.8982618e-02 -2.5048966e-02]]

CHGNet-predicted stress (GPa):
[[ 0.0063218   0.12258148 -0.35847756]
 [ 0.12258057 -0.9777256   0.18252584]
 [-0.3584779   0.18252821 -0.04905704]]

CHGNet-predicted magmom (mu_B):
[3.7762773  3.7847567  3.7762296  3.7847404  0.07918462 0.07919306
 0.07918471 0.07919323]



## Molecular Dynamics

In [3]:
from chgnet.model.model import CHGNet
from chgnet.model.dynamics import MolecularDynamics
from pymatgen.core import Structure
import warnings
warnings.filterwarnings("ignore", module="pymatgen")
warnings.filterwarnings("ignore", module="ase")

structure = Structure.from_file('../example_crystal.cif')
chgnet = CHGNet.load()

md = MolecularDynamics(
    atoms=structure,
    model=chgnet,
    ensemble="nvt",
    temperature=1000,  # in K
    timestep=2,  # in femto-seconds
    trajectory="md_out.traj",
    logfile="md_out.log",
    loginterval=100,
)
md.run(50)  # run a 0.1 ps MD simulation

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet will run on cuda
NVT-Berendsen-MD created


MD outputs are saved to the ASE trajectory file, to visualize the MD trajectory and magnetic moments after the MD run:

In [4]:
from ase.io.trajectory import Trajectory
from pymatgen.io.ase import AseAtomsAdaptor
from chgnet.utils import solve_charge_by_mag

traj = Trajectory("md_out.traj")
mag = traj[-1].get_magnetic_moments()

# get the non-charge-decorated structure
structure = AseAtomsAdaptor.get_structure(traj[-1])
print(structure)

# get the charge-decorated structure
struct_with_chg = solve_charge_by_mag(structure)
print(struct_with_chg)

Full Formula (Fe4 O4)
Reduced Formula: FeO
abc   :   3.102294   5.375696   5.443817
angles: 110.556657  91.096992  89.946600
pbc   :       True       True       True
Sites (8)
  #  SP           a         b          c    final_magmom  momenta
---  ----  --------  --------  ---------  --------------  ---------------
  0  Fe    0.5       0          0.5            3.77628    [0.0, 0.0, 0.0]
  1  Fe    0         0.5        0.5            3.78476    [0.0, 0.0, 0.0]
  2  Fe    0         0          0              3.77623    [0.0, 0.0, 0.0]
  3  Fe    0.500001  0.5       -0              3.78474    [0.0, 0.0, 0.0]
  4  O     0.968291  0.251676   0.742066       0.0791846  [0.0, 0.0, 0.0]
  5  O     0.468313  0.251673   0.242068       0.0791931  [0.0, 0.0, 0.0]
  6  O     0.031709  0.748324   0.257934       0.0791848  [0.0, 0.0, 0.0]
  7  O     0.531686  0.748327   0.757932       0.0791932  [0.0, 0.0, 0.0]
None


/tmp/ipykernel_112783/3845493673.py:13: UserWarning: Failed to solve oxidation state
  struct_with_chg = solve_charge_by_mag(structure)


## Model Training

In [ ]:
from chgnet.data.dataset import StructureData, get_train_val_test_loader
from chgnet.trainer import Trainer

dataset = StructureData(
    structures=list_of_structures,
    energies=list_of_energies,
    forces=list_of_forces,
    stresses=list_of_stresses,
    magmoms=list_of_magmoms,
)
train_loader, val_loader, test_loader = get_train_val_test_loader(
    dataset, batch_size=32, train_ratio=0.9, val_ratio=0.05
)
trainer = Trainer(
    model=chgnet,
    targets="efsm",
    optimizer="Adam",
    criterion="MSE",
    learning_rate=1e-2,
    epochs=50,
    use_device="cuda",
)

trainer.train(train_loader, val_loader, test_loader)